In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
sentence = """if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.""".split()
print(sentence)


['if', 'you', 'want', 'to', 'build', 'a', 'ship,', "don't", 'drum', 'up', 'people', 'together', 'to', 'collect', 'wood', 'and', "don't", 'assign', 'them', 'tasks', 'and', 'work,', 'but', 'rather', 'teach', 'them', 'to', 'long', 'for', 'the', 'endless', 'immensity', 'of', 'the', 'sea.']


In [3]:
vocab = sorted(list(set(sentence)))
print(vocab)
vocab_size = len(vocab)
print(vocab_size)

['a', 'and', 'assign', 'build', 'but', 'collect', "don't", 'drum', 'endless', 'for', 'if', 'immensity', 'long', 'of', 'people', 'rather', 'sea.', 'ship,', 'tasks', 'teach', 'the', 'them', 'to', 'together', 'up', 'want', 'wood', 'work,', 'you']
29


In [4]:
# enumerate(vocab, 1) : 인덱스가 1부터 시작됨
word_to_index = {c:i for i, c in enumerate(vocab, 1)}
print(word_to_index)

# {'unk' : 0} 을 추가함
word_to_index['unk'] = 0
print(word_to_index)
print(word_to_index['and'])
print(word_to_index['for'])

{'a': 1, 'and': 2, 'assign': 3, 'build': 4, 'but': 5, 'collect': 6, "don't": 7, 'drum': 8, 'endless': 9, 'for': 10, 'if': 11, 'immensity': 12, 'long': 13, 'of': 14, 'people': 15, 'rather': 16, 'sea.': 17, 'ship,': 18, 'tasks': 19, 'teach': 20, 'the': 21, 'them': 22, 'to': 23, 'together': 24, 'up': 25, 'want': 26, 'wood': 27, 'work,': 28, 'you': 29}
{'a': 1, 'and': 2, 'assign': 3, 'build': 4, 'but': 5, 'collect': 6, "don't": 7, 'drum': 8, 'endless': 9, 'for': 10, 'if': 11, 'immensity': 12, 'long': 13, 'of': 14, 'people': 15, 'rather': 16, 'sea.': 17, 'ship,': 18, 'tasks': 19, 'teach': 20, 'the': 21, 'them': 22, 'to': 23, 'together': 24, 'up': 25, 'want': 26, 'wood': 27, 'work,': 28, 'you': 29, 'unk': 0}
2
10


In [5]:
# 하이퍼파라미터 설정
hidden_size = vocab_size + 1
sequence_length = 3
learning_rate = 0.1
print(hidden_size)


30


In [6]:
def build_data(sentence, word_to_index):
    encoded = [word_to_index[token] for token in sentence]
    print(encoded)
    input_seq = encoded[:-1]
    label_seq = encoded[1:]
    print(label_seq)
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)
    label_seq = torch.LongTensor(label_seq).unsqueeze(0)
    return input_seq, label_seq

In [7]:
X, Y = build_data(sentence, word_to_index)
print(X)
print(Y)
print(X.size(), Y.size())

[11, 29, 26, 23, 4, 1, 18, 7, 8, 25, 15, 24, 23, 6, 27, 2, 7, 3, 22, 19, 2, 28, 5, 16, 20, 22, 23, 13, 10, 21, 9, 12, 14, 21, 17]
[29, 26, 23, 4, 1, 18, 7, 8, 25, 15, 24, 23, 6, 27, 2, 7, 3, 22, 19, 2, 28, 5, 16, 20, 22, 23, 13, 10, 21, 9, 12, 14, 21, 17]
tensor([[11, 29, 26, 23,  4,  1, 18,  7,  8, 25, 15, 24, 23,  6, 27,  2,  7,  3,
         22, 19,  2, 28,  5, 16, 20, 22, 23, 13, 10, 21,  9, 12, 14, 21]])
tensor([[29, 26, 23,  4,  1, 18,  7,  8, 25, 15, 24, 23,  6, 27,  2,  7,  3, 22,
         19,  2, 28,  5, 16, 20, 22, 23, 13, 10, 21,  9, 12, 14, 21, 17]])
torch.Size([1, 34]) torch.Size([1, 34])


In [8]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super().__init__()
        
        # embedding layer
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=input_size)
        #RNN 층
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=batch_first)

        #출력층

        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        # embedding 층 : 크기변화(배치크기, 시퀀스길이) => (배치크기, 시퀀스길이, 임베딩차원)
        output = self.embedding_layer(x)
        # RNN층 : 크기 변화(배치크기, 시퀀스길이, 임베딩차원) => 
        # ouput : (배치크기, 시퀀스길이, 은닉층크기)
        # hidden : (1, 배치크기, 은닉층 크기)
        output, hidden = self.rnn(output)

        # 출력층 : 크기변화(배치크기, 시퀀스길이, 은닉층 크기)
        # => (배치크기, 시퀀스길이, 단어장 크기)
        output = self.fc(output)

        # 크기 변화 : (배치크기, 시퀀스 길이, 단어장 크기) => (배치크기*시퀀스길이, 단어장크기)
        return output.view(-1, output.size(2))



In [9]:
model = Net(vocab_size+1, sequence_length, hidden_size, batch_first=True)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
index_to_word = {v: k for k, v in word_to_index.items()}
print(index_to_word)
print(word_to_index)


{1: 'a', 2: 'and', 3: 'assign', 4: 'build', 5: 'but', 6: 'collect', 7: "don't", 8: 'drum', 9: 'endless', 10: 'for', 11: 'if', 12: 'immensity', 13: 'long', 14: 'of', 15: 'people', 16: 'rather', 17: 'sea.', 18: 'ship,', 19: 'tasks', 20: 'teach', 21: 'the', 22: 'them', 23: 'to', 24: 'together', 25: 'up', 26: 'want', 27: 'wood', 28: 'work,', 29: 'you', 0: 'unk'}
{'a': 1, 'and': 2, 'assign': 3, 'build': 4, 'but': 5, 'collect': 6, "don't": 7, 'drum': 8, 'endless': 9, 'for': 10, 'if': 11, 'immensity': 12, 'long': 13, 'of': 14, 'people': 15, 'rather': 16, 'sea.': 17, 'ship,': 18, 'tasks': 19, 'teach': 20, 'the': 21, 'them': 22, 'to': 23, 'together': 24, 'up': 25, 'want': 26, 'wood': 27, 'work,': 28, 'you': 29, 'unk': 0}


In [11]:
outputs = model(X)
# print(outputs)
print(outputs.size())

torch.Size([34, 30])


In [12]:
decode = lambda y:[index_to_word.get(x) for x in y]

In [13]:
for step in range(500):
    optimizer.zero_grad()
    outputs = model(X)
    loss = loss_func(outputs, Y.view(-1))
    loss.backward()
    optimizer.step()

    pred=outputs.softmax(-1).argmax(-1).tolist()
    
    if step%10 ==0:
        print(f'{step}, {pred},\n { ' '.join(['if'] + decode(pred))}')

0, [11, 21, 16, 21, 11, 11, 26, 11, 21, 2, 11, 11, 21, 11, 11, 26, 16, 21, 8, 11, 26, 26, 16, 21, 8, 8, 26, 11, 21, 11, 21, 11, 11, 11],
 if if the rather the if if want if the and if if the if if want rather the drum if want want rather the drum drum want if the if the if if if
10, [29, 26, 23, 4, 1, 18, 7, 8, 25, 15, 24, 23, 6, 27, 2, 7, 3, 22, 19, 2, 28, 5, 16, 20, 22, 23, 13, 10, 21, 9, 12, 14, 21, 17],
 if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
20, [29, 26, 23, 4, 1, 18, 7, 8, 25, 15, 24, 23, 6, 27, 2, 7, 3, 22, 19, 2, 28, 5, 16, 20, 22, 23, 13, 10, 21, 9, 12, 14, 21, 17],
 if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
30, [29, 26, 23, 4, 1, 18, 7, 8, 25, 15, 24, 23, 6, 27, 2, 7, 3, 22, 19, 2, 28, 5, 16, 20, 22,

In [20]:
result = model(X).argmax(-1).tolist()
print(decode(result))

['you', 'want', 'to', 'build', 'a', 'ship,', "don't", 'drum', 'up', 'people', 'together', 'to', 'collect', 'wood', 'and', "don't", 'assign', 'them', 'tasks', 'and', 'work,', 'but', 'rather', 'teach', 'them', 'to', 'long', 'for', 'the', 'endless', 'immensity', 'of', 'the', 'sea.']
